# Data inspection for ASD
Inspecting data that was chosen to be used for estimating the ASD. 1264315666 ± 256s was chosen because this time is relatively quiet and close to the event time.

Parameters:
- 4s duration FFTs with 2s overlap
- 512s data
- median ASD

Note: use igwn-py39 env [or any other IGWN-approved env] at a LIGO cluster

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from gwpy.timeseries import TimeSeries
from gwpy.plot import Plot

In [ ]:
gps_event = 1264316116
gps_asd = gps_event - 450
asd_win = 256
start_gps = gps_asd - asd_win
end_gps = gps_asd + asd_win

frate = 512

## Get data

In [ ]:
frame = 'L1_HOFT_C01'

channels = ['L1:DCS-CALIB_STRAIN_CLEAN_C01',
            'L1:LSC-POP_A_RF45_I_ERR_DQ',
            'L1:LSC-POP_A_RF45_Q_ERR_DQ',
            'L1:LSC-POP_A_RF9_I_ERR_DQ']

tseries = []
for channel in channels:
    tseries.append(TimeSeries.get(channel=channel, start=start_gps, end=end_gps, frametype=frame).resample(frate))

## Plot tseries and oscans

In [ ]:
# strain
plot = tseries[0].plot()
plot.show()

oscan = tseries[0].q_transform(qrange=(10,20))
plot_orig = oscan.plot()
ax = plot_orig.gca()
ax.set_yscale('log')
ax.set_ylim(10,frate/2)
ax.colorbar(cmap='viridis', label='Normalized energy',vmin=0,vmax=25)
ax.grid(alpha=0.6)

# aux
plot = tseries[1].plot()
plot.show()

oscan = tseries[1].q_transform(qrange=(10,20))
plot_orig = oscan.plot()
ax = plot_orig.gca()
ax.set_yscale('log')
ax.set_ylim(10,frate/2)
ax.colorbar(cmap='viridis', label='Normalized energy',vmin=0,vmax=25)
ax.grid(alpha=0.6)

## Show median ASDs

In [ ]:
asds = []

for idx in range(len(channels)):
    
    median_asd = tseries[idx].asd(4, 2, method='median')
    
    # inflate the ASD below 10Hz
    cutoff_idx = np.argmin(np.abs(median_asd.frequencies.value - 10))
    max_val = np.max(median_asd)
    median_asd[0:cutoff_idx] = max_val
    
    asds.append(median_asd)

    plot = Plot()
    ax = plot.add_subplot(
        xscale='log', xlim=(1, frate/2), xlabel='Frequency [Hz]',
        yscale='log',
        ylabel=r'Noise [1/$\sqrt{\mathrm{Hz}}$]',
    )
    ax.plot(median_asd.frequencies, median_asd, color='gwpy:ligo-livingston')